In [1]:
import MetaTrader5 as mt5
import pandas as pd
import pandas_ta as ta
import time
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from pylab import mpl, plt
import plotly.express as px

In [2]:
plt.style.use('seaborn')
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'

In [3]:
df = pd.read_csv('../IBRA_D.csv', sep=';')
df.rename(columns={"C�digo": "CODIGO", "A��o": "AÇAO"}, inplace=True)
df = df.iloc[:-2, :]
df['CODIGO']

0      RRRP3
1      TTEN3
2      ABCB4
3      AERI3
4      AESB3
       ...  
205    VULC3
206    WEGE3
207    WEST3
208    WIZS3
209    YDUQ3
Name: CODIGO, Length: 210, dtype: object

In [4]:
Symbols = df['CODIGO']
Symbols

0      RRRP3
1      TTEN3
2      ABCB4
3      AERI3
4      AESB3
       ...  
205    VULC3
206    WEGE3
207    WEST3
208    WIZS3
209    YDUQ3
Name: CODIGO, Length: 210, dtype: object

In [5]:
ativos = [tick for tick in Symbols]
ativos

['RRRP3',
 'TTEN3',
 'ABCB4',
 'AERI3',
 'AESB3',
 'ALSO3',
 'AALR3',
 'ALLD3',
 'ALPA4',
 'ALUP11',
 'ABEV3',
 'AMBP3',
 'AMER3',
 'ANIM3',
 'ARZZ3',
 'ARML3',
 'ASAI3',
 'AZUL4',
 'B3SA3',
 'BMGB4',
 'BIDI11',
 'BPAN4',
 'BRSR6',
 'BBSE3',
 'BMOB3',
 'BKBR3',
 'BLAU3',
 'BOAS3',
 'BRML3',
 'BRPR3',
 'BBDC3',
 'BBDC4',
 'BRAP4',
 'BBAS3',
 'AGRO3',
 'BRKM5',
 'BRFS3',
 'BPAC11',
 'CXSE3',
 'CAML3',
 'CRFB3',
 'CBAV3',
 'CCRO3',
 'CEAB3',
 'CMIG3',
 'CMIG4',
 'CESP6',
 'CIEL3',
 'CLSA3',
 'COGN3',
 'CSMG3',
 'CPLE3',
 'CPLE6',
 'CSAN3',
 'CPFE3',
 'CMIN3',
 'CARD3',
 'CURY3',
 'CVCB3',
 'CYRE3',
 'DASA3',
 'DXCO3',
 'DEXP3',
 'PNVL3',
 'DIRR3',
 'ECOR3',
 'ELET3',
 'ELET6',
 'EMBR3',
 'ENAT3',
 'ENBR3',
 'ENGI11',
 'ENEV3',
 'EGIE3',
 'ENJU3',
 'EQTL3',
 'ESPA3',
 'EVEN3',
 'EZTC3',
 'FESA4',
 'FLRY3',
 'GFSA3',
 'GGBR4',
 'GOAU4',
 'GETT11',
 'GOLL4',
 'GGPS3',
 'GRND3',
 'GMAT3',
 'NTCO3',
 'SBFG3',
 'SOMA3',
 'GUAR3',
 'HAPV3',
 'HBOR3',
 'HBSA3',
 'HYPE3',
 'IGTI11',
 'PARD3',
 'ME

In [6]:
def get_ohlc(ativo, timeframe, n=5):
    ativo = mt5.copy_rates_from_pos(ativo, timeframe, 0, n)
    ativo = pd.DataFrame(ativo)
    ativo['time'] = pd.to_datetime(ativo['time'], unit='s')
    ativo.set_index('time', inplace=True)
    return ativo

In [7]:
# calculo percentil para determinar intervalo de confiança

def percentil(data, up=90, down=10):
    x = data
    x = x.dropna()
    array_x = np.array(x)
    percentil_up = np.percentile(array_x, up)
    percentil_down = np.percentile(array_x, down)
    
    return [percentil_up, percentil_down]

In [8]:
def caroBarato(tick_ativo, up, down, dias):
    # coleta de dados
    if not mt5.initialize():
        print('initialize() falhou')
        mt5.shutdown()

    df = get_ohlc(tick_ativo, mt5.TIMEFRAME_D1, dias)

    # criação médias
    df['M20'] = ta.sma(df['close'], length=20)
    df['M50'] = ta.sma(df['close'], length=50)
    df['M200'] = ta.sma(df['close'], length=200)

    #criação distancia média
    df['Dist20'] = df['close'] - df['M20'] 
    df['Dist50'] = df['close'] - df['M50']
    df['Dist200'] = df['close'] - df['M200']
    
    # filtro
    df = df[df['real_volume'] > 0]

    # calculo percentil das distancia da média 
    # return percentil_up, percentil_down
    per_m20 = percentil(df['Dist20'], up, down)
    per_m50 = percentil(df['Dist50'], up, down)
    per_m200 = percentil(df['Dist200'], up, down)

    # transformação em um array
    
    # variavel temporaria
    valor = 0

    # logica
    # logica valor = +1 +2 +4 or -1 -2 -4 para diferenciar qual média aponta distancia
    if df['Dist20'][-1] > per_m20[0]:
        valor += 1
    
    if df['Dist50'][-1] > per_m50[0]:
        valor += 2
    
    if df['Dist200'][-1] > per_m200[0]:
        valor += 4
    
    if df['Dist20'][-1] < per_m20[1]:
        valor -= 1
    
    if df['Dist50'][-1] < per_m50[1]:
        valor -= 2
    
    if df['Dist200'][-1] < per_m200[1]:
        valor -= 4

    return valor

In [9]:
caroBarato('PETR4', 95, 5, 500)

0

In [10]:
dicionarioCarobarato = {}
for ativo in ativos:
    try:
        dicionarioCarobarato[ativo] = caroBarato(ativo, 95, 5, 500)
        print(f'COLETA DE {ativo}')
    except:
        print(f'{ativo} SEM COLETA')
        #dicionarioCarobarato[ativo] = 'NAN'
dicionarioCarobarato = pd.DataFrame.from_dict(dicionarioCarobarato, orient='index').reset_index()

COLETA DE RRRP3
TTEN3 SEM COLETA
COLETA DE ABCB4
COLETA DE AERI3
COLETA DE AESB3
COLETA DE ALSO3
COLETA DE AALR3
COLETA DE ALLD3
COLETA DE ALPA4
COLETA DE ALUP11
COLETA DE ABEV3
COLETA DE AMBP3
COLETA DE AMER3
COLETA DE ANIM3
COLETA DE ARZZ3
ARML3 SEM COLETA
COLETA DE ASAI3
COLETA DE AZUL4
COLETA DE B3SA3
COLETA DE BMGB4
COLETA DE BIDI11
COLETA DE BPAN4
COLETA DE BRSR6
COLETA DE BBSE3
COLETA DE BMOB3
COLETA DE BKBR3
COLETA DE BLAU3
COLETA DE BOAS3
COLETA DE BRML3
COLETA DE BRPR3
COLETA DE BBDC3
COLETA DE BBDC4
COLETA DE BRAP4
COLETA DE BBAS3
COLETA DE AGRO3
COLETA DE BRKM5
COLETA DE BRFS3
COLETA DE BPAC11
CXSE3 SEM COLETA
COLETA DE CAML3
COLETA DE CRFB3
CBAV3 SEM COLETA
COLETA DE CCRO3
COLETA DE CEAB3
COLETA DE CMIG3
COLETA DE CMIG4
COLETA DE CESP6
COLETA DE CIEL3
CLSA3 SEM COLETA
COLETA DE COGN3
COLETA DE CSMG3
COLETA DE CPLE3
COLETA DE CPLE6
COLETA DE CSAN3
COLETA DE CPFE3
COLETA DE CMIN3
COLETA DE CARD3
COLETA DE CURY3
COLETA DE CVCB3
COLETA DE CYRE3
COLETA DE DASA3
COLETA DE DXCO3


In [11]:
dicionarioCarobarato.columns = ["Ticker", "Valor"]
dicionarioCarobarato

,Ticker,Valor
0,RRRP3,0
1,ABCB4,0
2,AERI3,0
3,AESB3,0
4,ALSO3,0
...,...,...
188,VULC3,-4
189,WEGE3,0
190,WEST3,0
191,WIZS3,0


In [12]:
ativos_caros = dicionarioCarobarato[dicionarioCarobarato['Valor'] > 1]
ativos_caros.sort_values(by='Valor')

,Ticker,Valor
52,CURY3,2
162,SLCE3,2
22,BMOB3,4
69,ENJU3,4
133,ORVR3,4
136,PETR3,4
140,PLPL3,4
153,SAPR11,4
46,CPLE3,6
47,CPLE6,6


In [13]:
ativos_baratos = dicionarioCarobarato[dicionarioCarobarato['Valor'] < -1]
ativos_baratos.sort_values(by='Valor')

,Ticker,Valor
34,BRFS3,-7
7,ALPA4,-6
151,RAIL3,-6
9,ABEV3,-4
19,BPAN4,-4
41,CMIG4,-4
63,EMBR3,-4
96,RANI3,-4
97,IRBR3,-4
101,JALL3,-4


In [14]:
# colocar filtro de liquidez
# colocar coleta yahoo finace
# colocar Scatterplot 

In [15]:
# filtro diferente de 0
dicionarioCarobarato = dicionarioCarobarato[dicionarioCarobarato['Valor'] != 0]

In [16]:
dicionarioCarobarato = dicionarioCarobarato.sort_values(by='Valor')

In [22]:
fig = px.scatter_matrix(dicionarioCarobarato)
fig.show()

fig2 = px.scatter(dicionarioCarobarato, x='Ticker', y='Valor',marginal_x='box', marginal_y='violin',  hover_name='Ticker')
fig2.show()